In [62]:
import sys 
sys.path.append('/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main')

In [63]:
import numpy as np 
import math 

import parameters as pr 
import sys 

from numba import jit 
from numba.extending import overload        

# metropolis sampling and integral approximation 

%run /home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/auxiliaryFunctions_noMath.ipynb 
%run /home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/functions_module.ipynb
%run /home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main/gradientDescent_module.ipynb



# Possible solutions to Numba compatibility 

convert formula to fully python: e.g. ``math.sqrt(x) -> (x)**(1/2) and math.exp(x) -> 2.717**(x)``


# User definable variables 

**Input:** Initial guess for parameters: 

In [64]:
parametersList = [0.2, 2, 3] 

**Input:** Number of parallel configuration subspaces (parallel configuration spaces)

In [65]:
M =1

**Input**: Filenames for the Python-compatible formulae of probability function and local energy (Supposed to be in the default folder "Formulae_PythonCompatible"  )

In [66]:
probabilityExprPath = "probabilityFormula_pythonCompatible"


localEnergyExprPath = "localEnergy_pythonCompatible"

**Input**: Samples/sampling settings in Metropolis sampling

In [67]:
coordinateValueRange= (-10, 10)
numberOfParticles = 3
numberOfConfig = 1
numberOfIterations = 1            

# Preparing local energy functions and probability functions 

## Load in probability and local energy expressions 

In [68]:
# default folder to the Python-compatible formulae 
defaultFolderPathForFormulae = "/home/degnaiyu/Työpöytä/kanditutkielma/kandityo_koodit/main"+ "/"+ "Formulae_PythonCompatible/"    

In [69]:
# global variables: used in probability and local energy functions definition

probabilityExpr = readFileToString(defaultFolderPathForFormulae + probabilityExprPath)  

localEnergyExpr = readFileToString(defaultFolderPathForFormulae + localEnergyExprPath)

In [70]:
probabilityExpr

'math.exp(-2*A3*math.sqrt(x2**2 + y2**2 + z2**2) -      (2*A2*(1 - math.exp           (-math.sqrt(abs(x1 - x2)**2 + abs(y1 - y2)**2 + abs(z1 - z2)**2)/             math.sqrt(A2))))/      math.sqrt(abs(x1 - x2)**2 + abs(y1 - y2)**2 + abs(z1 - z2)**2) -      (2*A1*(1 - math.exp           (-math.sqrt(abs(x1 - x3)**2 + abs(y1 - y3)**2 + abs(z1 - z3)**2)/             (math.sqrt(2)*math.sqrt(A1)))))/      math.sqrt(abs(x1 - x3)**2 + abs(y1 - y3)**2 + abs(z1 - z3)**2) -      (2*A2*(1 - math.exp           (-math.sqrt(abs(x2 - x3)**2 + abs(y2 - y3)**2 + abs(z2 - z3)**2)/             math.sqrt(A2))))/      math.sqrt(abs(x2 - x3)**2 + abs(y2 - y3)**2 + abs(z2 - z3)**2))*  (2*math.exp(-0.5*(A3*math.sqrt(x1**2 + y1**2 + z1**2)) -          A3*math.sqrt(x3**2 + y3**2 + z3**2)) -      2*math.exp(-(A3*math.sqrt(x1**2 + y1**2 + z1**2)) -          (A3*math.sqrt(x3**2 + y3**2 + z3**2))/2.) -      math.exp(-0.5*(A3*math.sqrt(x1**2 + y1**2 + z1**2)) -          A3*math.sqrt(x3**2 + y3**2 + z3**2))*math.sqrt(

In [71]:
localEnergyExpr[:50] 

'(math.exp(A3*math.sqrt(x2**2 + y2**2 + z2**2) +   '

## Find variables in the expressions

In [72]:
# find coordinates and parameters
coordinates, parameterStringList  = findVariables(localEnergyExpr)

# regrouping coordinates 
coordinates = regroupToCoordinateTriple_findVariables(coordinates)


# for functions usage, define list of ordered variables and parameters
mapping =  coordinates + parameterStringList  # global variable: used in probability and local energy functions definition


In [73]:
coordinates, parameterStringList, mapping 

(['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3'],
 ['A1', 'A2', 'A3'],
 ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2', 'A3'])

### Error check: check number of initial guess is same as number of parameters found

In [74]:
check_numberOfIni_VS_numberOfFound_parameters(parametersList, parameterStringList) # raises exception if error occurs, program stopped

## Define probability and local energy functions

In [75]:
@jit
def probabilityFunc(R: np.ndarray, parameters: list, expr: str = probabilityExpr, mapping: list = mapping) -> float: 
    
    '''
    psi^2. Probability function. Dependent on configuration and parameters. 
    
    Parameters
    ----------
        R: ordered current configuration e.g. 
            array([[-0.5583685 , -0.04608995,  0.15500853],
                 [ 0.66255653,  0.66301583, -0.85159876]])
             each row representing one single particle's position
        
        parameters: ordered list of parameters as numbers 
        
        expr: formula string. Default to the global variable 
        
        mapping: ordered coordinates and parameters in a list: 
            ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2']. Default to the global variable 
        
    Return
    ------
        Probability function evaluated at certain configuration and parameters' values
    '''
    
    
    
    
    R_flattened = R.flatten() # make current configuration to vector form 
    R_and_parameters = np.append(R_flattened, parameters) # ordered coordinates, and parameters at the tail of the vector 
    
    
    if len(R_and_parameters) != len(mapping): 
        raise Exception("R and parameters not same length as that of mapping!")
    
    
    # mapping dictionary for evaluation of the expression
    localDict = dict(zip(mapping, R_and_parameters))

    
    
    return eval(expr, {'math': math}, localDict)
    
    
    
    

@jit
def localEnergyFunc(R: np.ndarray, parameters: list, expr: str = localEnergyExpr, mapping: list = mapping ) -> float: 
    '''
    
    H*psi/psi. Local energy.  Dependent on configuration and parameters. 
    
    Parameters
    ----------
        R: current configuration
        
        parameters: list of parameters as numbers 
        
        expr: formula string 
        
        mapping: ordered coordinates and parameters in a list: 
            ['x1', 'y1', 'z1', 'x2', 'y2', 'z2', 'x3', 'y3', 'z3', 'A1', 'A2']
            
    Return
    ------
        Local energy evaluated at certain configuration and parameters' values
    
    '''
    
    
    R_flattened = R.flatten() # make current configuration to vector form 
    R_and_parameters = np.append(R_flattened, parameters) # ordered coordinates, and parameters at the tail of the vector 
    
    
    if len(R_and_parameters) != len(mapping): 
        raise Exception("R and parameters not same length as that of mapping!")
    
    
    # mapping dictionary for evaluation of the expression
    localDict = dict(zip(mapping, R_and_parameters))

    
    
    
    
    return eval(expr, {'math': math}, localDict)
    
    
    

# Metropolis sampling and local energy estimations 

In [76]:
energyDict = {'Energy': [], 
             'Variance': [], 
             'std error': []}  # to append list of 

## Configuration subspaces: getting samples for energy

In [77]:

# to save list of average energies from each configuration subspace 
energyInConfigSpaceList = []


# for each configuration subspace 
for i in range(M): 
    
    # generating initial Metropolis samples and Thermalisation
    configSubspace_Metropolis = metropolisSamplingFunction( coordinateValueRange=  coordinateValueRange, 
                                                           numberOfParticles = numberOfParticles,  
                                                           numberOfConfig = numberOfConfig, 
                                                           probabilityFunction=probabilityFunc, 
                                                           numberOfIterations = numberOfIterations, 
                                                           params = parametersList)
    
    
    energyInConfigSubspaceList = []           # for saving the energy values in this subspace 
    
    
 
    # energy computation for each Metropolis suggestion
    # for each configuration 
    for configIndex in range(configSubspace_Metropolis.shape[1]): 
        # current configuration in format: 
        #  array([[-0.5583685 , -0.04608995,  0.15500853],
       # [ 0.66255653,  0.66301583, -0.85159876]])
        # each row representing one particle's position  
        currentConfig = configSubspace_Metropolis[:, configIndex, :]   
        
        # for each particle's displacement, compute energy 
        for nthParticleIndex in range(len(currentConfig)): 
            # making Metropolis-suggestion on single particle
            updatedConfig = metropolisStepSuggestion(configuration= currentConfig, 
                                                    nthParticle= nthParticleIndex, 
                                                    probabilityFunction=probabilityFunc, 
                                                    params=parametersList)
            
            # update configuration subspace 
            configSubspace_Metropolis[:, configIndex, :] = updatedConfig
            
            
            # compute energy 
            energyInConfigSubspace = monteCarloIntegrationFunction(samples= configSubspace_Metropolis, 
                                                                     localEnergyFunction=localEnergyFunc, 
                                                                     params=parametersList)
            
            # append to the list 
            energyInConfigSubspaceList.append(energyInConfigSubspace)
        
    
    # compute mean energy of this subspace and append it to the whole space list 
    energyInConfigSpaceList.append(np.mean(energyInConfigSubspaceList))
    
    print(f'{i}.loop completed')
    
            
            
        
        
    

NumbaWarning: Failed in object mode pipeline (step: object mode frontend)
Failed in object mode pipeline (step: object mode backend)
Function "metropolisSamplingFunction" was compiled in object mode without forceobj=True, but has lifted loops.

File "../../../../../../tmp/ipykernel_4218/418465071.py", line 34:
<source missing, REPL/exec in use?>


## compute energy approximation 

In [ ]:
# compute mean 
energyMean =np.mean(energyInConfigSpaceList)


# compute variance and std error of mean, if there are more than 1 samples 
if len(energyInConfigSpaceList)> 1: 
    # sample variance (N-1)
    energyVar =np.var(energyInConfigSpaceList, ddof= 1)

    # std error of mean 
    energyError = energyVar/len(energyInConfigSpaceList)
    
    
    # printing the values 
    print('Mean:', energyMean)
    print('Variance:', energyError)
    print('Std error of mean:', energyError)
    
    # saving values 
    energyDict['Energy'].append(energyMean)
    energyDict['Variance'].append(energyVar)
    energyDict['std error'].append(energyError)
else: 
    print('Variance and std error of mean cannot be calculated: There is only 1 sample')
    print()
    print(energyMean)
    
    # saving values 
    energyDict['Energy'].append(energyMean)
    energyDict['Variance'].append(0)
    energyDict['std error'].append(0)
    
    


# Optimization of parameters 

In [ ]:
parametersList

In [ ]:
# generate new Metropolis samples
metropolisSamples_Opt = metropolisSamplingFunction( coordinateValueRange=  coordinateValueRange, 
                                                           numberOfParticles = numberOfParticles,  
                                                           numberOfConfig = numberOfConfig, 
                                                           probabilityFunction=probabilityFunc, 
                                                           numberOfIterations = numberOfIterations, 
                                                           params = parametersList)



# optimizing 
parametersList = gradientDescentFunction(monteCarloIntegrationFunc=monteCarloIntegrationFunction, 
                                        function=localEnergyFunc, 
                                        learning_rate= 0.001, 
                                        samples= metropolisSamples_Opt, 
                                        parameters=parametersList)

In [ ]:
parametersList 